Решение задачи теории расписания методом полного перебора и методом восхождения на холм

In [11]:
import itertools
import sys
import numpy as np
import random
import time

F1=[]
for i in range(100):
        F1.append(0.0)
F2=[]
for i in range(100):
        F2.append(0.0)


choise2=int(input("Cгенерировать новые данные? \n 1. Да \n 2. Нет"))

n = int(input("количество заявок:",))                            # количество заявок

N=0

y= []
for i in range(n):
    y.append(0.0)

if choise2==1:                                                   # если сгенерировать заново

    f=open("data.txt",'w')                                       # открытие для записи
    #f.write("x \t\t t \t\t d \t\t c \t\t D \t\t y \n")          # шапка

    for rep in range(100):
        for i in range(n): 
            
            t = round(random.random()*(10)+8,2)
            d = round(random.random()*(10)+6,2)
            c = round(random.random(),2)
            D = round(random.random()*(2),2)

            f.write(str(t) + '\t' + str(d) + '\t' + str(c)+ '\t' + str(D))   # получили строку
            f.write('\n') 
    f.close()                           # закрыли файл


t=[]                                    # длительность выполнения
c=[]                                    # время поступления   
d=[]                                    # стоимость ожидания обслуживания           
D=[]                                    # директивные сроки 


def txt(k,kol):
    file = open("data.txt", "r")      # открытие файла для чтения
    lines = file.readlines()          # считываем все строки
    #k=0                              # динамический счетчик по длине строки
    #kol=0                            # проверка, какие брать строки из файла
    check=0                           # проверка, в какой массив заносить данные

    for line in lines:                      # итерация по строкам
        while (kol<kol+n):                  # по 5 строк берем       
            l=line.strip().split()          # strip удаляет пробелы в начале и конце, split разделяет строку по пробелам
            for i in l:                    # по строке
                i=float(i)
                if check==0: t.append(i)
                if check==1: d.append(i)   
                if check==2: c.append(i)
                if check==3: D.append(i)
                check+=1
            k+=1
            check=0    
            break
    kol=kol+k
    file.close                               # закрываем файл


a = [int(input("введите заявку (начиная с 0): ")) for i in range(n) ]            # запрос заявок


def rasp(p1):

    global p_itog
    global begin_time
    global criterion

    min = sys.maxsize         # минимальное значение КРИТЕРИЯ
    begin_time=sys.maxsize    # время начала в составляемом расписании
              
    criterion=0               # сюда суммируем штраф каждой заявки из перестановки (получаем значение критерия) 

    for i in reversed(p1):    # по перестановке с конца

        if D[i] > begin_time:
            y[i] = begin_time
            x[i] = y[i]-t[i]
            begin_time = x[i]

            if x[i] < d[i]:
                if 0 >= d[i]-x[i]: phi = 0          # phi- максимум из 0 и разности d-x
                else: phi = c[i]*(d[i]-x[i])
                criterion += phi                    # добавили критерию штраф заявки

        if D[i] <= begin_time:
            y[i] = D[i]
            x[i] = y[i]-t[i]
            begin_time = x[i]

            if x[i] < d[i]:
                if 0 >= d[i]-x[i]: phi = 0          # phi- максимум из 0 и разности d-x
                else: phi = c[i]*(d[i]-x[i])
                criterion += phi                    # добавили критерию штраф заявки
    criterion=round(criterion,4)
    #print('criterion', criterion)


choise1= int(input("Выберите алгоритм: \n 1. Полный перебор \n 2. Метод восхождения на холм "))


start_time = time.time()
############################ Полный перебор ##############################

if choise1 == 1:
    k=0                       # динамический счетчик по длине строки
    kol=0                     # проверка, какие брать строки из файла
    
    txt(k,kol)
    
    min = sys.maxsize         # минимальное значение КРИТЕРИЯ
    begin_time=sys.maxsize    # время начала в составляемом расписании
    p_itog=[]                 # итоговое расписание
    criterion=0               # сюда суммируем штраф каждой заявки из перестановки (получаем значение критерия) 
    
    x=[]
    for i in range(n):
        x.append(0.0)

    for p1 in itertools.permutations(a, len(a)):        # использование класса генерации перестановок
        #print('-----------------------------')
        #print(p1)

        rasp(p1)

        if criterion < min:                             # изменили минимум,запомнили перестановку, при которой он получился
            min = criterion
            p_itog = p1
            min=round(min, 4)
            #print('min=', min)
        criterion = 0
        begin_time = sys.maxsize                        # время начала в составляемом расписании

    print('\n-----Answer-------')
    print('Полученное расписание: ', p_itog)
    print('Значение критерия: ', min)

######################################################################


###################### Метод восхождения на холм #####################

if choise1 == 2:

    k=0                       # динамический счетчик по длине строки
    kol=0                     # проверка, какие брать строки из файла

    txt(k,kol)

    min = sys.maxsize         # минимальное значение КРИТЕРИЯ
    begin_time=sys.maxsize    # время начала в составляемом расписании
    p_itog=[]                 # итоговое расписание
    criterion=0               # сюда суммируем штраф каждой заявки из перестановки (получаем значение критерия)
    Epsilon=[]                # окрестность
    list=[]                   # для синхронизации сортировки (храним тут все)
                     
    x=[]
    for i in range(n):
        x.append(0.0)


    choise= int(input("как сгенерировать начальную точку? \n 1. Случайно \n 2. С упорядочением"))

    if choise==1: p1=np.random.permutation(a)                     # стартовая перестановка рандомно
    if choise ==2:                                                # стартовая перестановка с учетом упорядочения по длительности выполнения заявок     
        
        list = zip(t,d,c,D,a)
        ls = sorted(list, key=lambda l:l[0], reverse=True)        # по t, т.к. l[0] в порядке неубывания
        t = [list[0] for list in ls]                              # разделяем обратно
        d = [list[1] for list in ls]
        c = [list[2] for list in ls]
        D = [list[3] for list in ls]
        a = [list[4] for list in ls]
        p1=a
    #print('Стартовая перестановка', p1)

    rasp(p1)

    if criterion <min:                              # изменили минимум,запомнили перестановку, при которой он получился
        min=criterion
        p_itog=p1
        min=round(min, 4)
        #print('min=',min)
    criterion=0
    begin_time=sys.maxsize                          # время начала в составляемом расписании


    ###################### Генерируем окрестность ####################

    def transpositions(p1):
        p2 = p1.copy()
        for i in range(len(p1)-1):
            for j in range(i+1, len(p1)):
                p2[j], p2[i] = p1[i], p1[j]
                #print(i,j,p2)
                p2 = p1.copy()
                Epsilon.append(p2)
    ################################################################## 

    min1=[]
    min1.append(True)
    chek=True

    while chek==True:
        min1.clear()
        transpositions(p1)

        #print ('----------------------')
        #print('Окрестность:', Epsilon)

        for i in range (len(Epsilon)):
            #print ('----------------------')
            #print('Neigbor:', Epsilon[i])
            
            rasp(Epsilon[i])

            if criterion <min:                          # изменили минимум,запомнили перестановку, при которой он получился
                min=criterion
                min=round(min, 4)
                p1=Epsilon[i]
                #print('min=', min)
                min1.append(True)
            else: min1.append(False)
    
            criterion=0
            begin_time=sys.maxsize                      # время начала в составляемом расписании
        Epsilon.clear()
    
        if True not in min1:
            chek=False
        #print ('----------------------')

        if chek==False:
            print ('----------------------')
            print ('Найдено локальное решение!')
            print ('Полученное расписание: ', p1)
            print ('Значение критерия: ', min) 
            print ('----------------------\n')    
            break
   
#########################################################################################################
print("--- %s seconds ---" % (time.time() - start_time)) 



-----Answer-------
Полученное расписание:  (7, 6, 4, 5, 9, 2, 8, 10, 1, 3, 0)
Значение критерия:  337.5276
--- 589.0353319644928 seconds ---


Время расчета подного перебора для длины 11:
-----Answer-------
Полученное расписание:  (7, 6, 4, 5, 9, 2, 8, 10, 1, 3, 0)
Значение критерия:  337.5276
--- 589.0353319644928 seconds ---